In [23]:
import pandas as pd
import requests
from io import StringIO

In [24]:
url = "https://db.netkeiba.com/race/201901010101"
pd.read_html(url, encoding='euc-jp')[0]

/Users/nasutougo/Desktop/racing_tip/practice/project_env/lib/python3.10/site-packages/bs4/__init__.py:228: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


,着 順,枠 番,馬 番,馬名,性齢,斤量,騎手,タイム,着差,ﾀｲﾑ 指数,...,上り,単勝,人 気,馬体重,調教 ﾀｲﾑ,厩舎 ｺﾒﾝﾄ,備考,調教師,馬主,賞金 (万円)
0,1,1,1,ゴルコンダ,牡2,54,ルメール,1:48.3,NaN,**,...,36.5,1.4,1,518(-16),NaN,NaN,NaN,[東] 木村哲也,サンデーレーシング,500.0
1,2,3,3,プントファイヤー,牡2,54,岩田康誠,1:50.1,大,**,...,38.2,3.5,2,496(-8),NaN,NaN,NaN,[東] 手塚貴久,嶋田賢,200.0
2,3,4,4,ラグリマスネグラス,牡2,51,団野大成,1:50.9,5,**,...,38.8,46.6,6,546(+6),NaN,NaN,NaN,[東] 藤沢和雄,多田信尊,130.0
3,4,8,9,キタノコドウ,牡2,51,菅原明良,1:51.5,3.1/2,**,...,38.5,56.8,7,458(-8),NaN,NaN,NaN,[東] 高木登,北所直人,75.0
4,5,5,5,ネモフィラブルー,牡2,54,川島信二,1:51.7,1.1/2,**,...,38.9,140.3,9,436(0),NaN,NaN,NaN,[西] 矢作芳人,青芝商事,50.0
5,6,8,8,マイネルラクスマン,牡2,54,丹内祐次,1:52.1,2.1/2,**,...,40.1,9.7,3,480(+8),NaN,NaN,NaN,[東] 金成貴史,サラブレッドクラブ・ラフィアン,NaN
6,7,2,2,サンモンテベロ,牝2,54,黛弘人,1:52.5,2.1/2,**,...,39.6,114.7,8,450(+2),NaN,NaN,NaN,[東] 中野栄治,加藤ステーブル,NaN
7,8,7,7,エスカレーション,牝2,54,藤岡佑介,1:52.5,アタマ,**,...,39.5,26.1,5,448(-4),NaN,NaN,NaN,[東] 高柳瑞樹,松田整二,NaN
8,9,6,6,セイウンジュリア,牝2,54,池添謙一,1:52.6,クビ,**,...,40.3,16.4,4,470(0),NaN,NaN,NaN,[西] 浅見秀一,西山茂行,NaN


01:札幌, 02:函館, 03:福島, 04:新潟, 05:東京, 06:中山, 07:中京, 08:京都, 09:阪神, 10:小倉

In [41]:
import time
from tqdm import tqdm

def scrape_race_results(race_id_list, pre_race_results = {}):
  # 辞書としてrace_resultsを初期化
  # race_results = {}
  race_results = pre_race_results
  for race_id in tqdm(race_id_list):
    if race_id in race_results.keys(): # 
      continue
    try:
      url = 'https://db.netkeiba.com/race/' + race_id
      response = requests.get(url)

      # response.contentから得られるバイナリデータを'euc-jp'でデコード
      content = response.content.decode('euc-jp')

      #StringIOを使用してデコードされたHTMLコンテンツをpandasに渡す
      df = pd.read_html(StringIO(content))[0]
      
      # race_idをキーとして結果を辞書に格納
      race_results[race_id] = df
      time.sleep(1)
    except IndexError: # テーブルがない場合のエラー処理
      continue
    except:
      break


  return race_results

In [10]:
# レースIDのリスト
race_id_list = ["201902010101", "201902010102", "201902010103"]

# 関数をテスト
test = scrape_race_results(race_id_list)

In [26]:
race_id_list = []
for place in range(1,11,1):
  for kai in range(1,6,1):
    for day in range(1,9,1):
      for r in range(1,13,1):
        race_id = "2019" + str(place).zfill(2) + str(kai).zfill(2) + str(day).zfill(2) + str(r).zfill(2)
        race_id_list.append(race_id)

In [27]:
len(race_id_list)

4800

In [43]:
test4 = scrape_race_results(race_id_list, test3)

100%|██████████| 4800/4800 [1:26:23<00:00,  1.08s/it]


In [47]:
for key in test4.keys():
  test4[key].index = [key] * len(test4[key])

In [51]:
results = pd.concat([test4[key] for key in test4.keys()], sort=False)

In [52]:
results.to_pickle('results.pickle')

,着 順,枠 番,馬 番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人 気,馬体重,調教師
201901010101,1,1,1,ゴルコンダ,牡2,54.0,ルメール,1:48.3,NaN,1.4,1.0,518(-16),[東] 木村哲也
201901010101,2,3,3,プントファイヤー,牡2,54.0,岩田康誠,1:50.1,大,3.5,2.0,496(-8),[東] 手塚貴久
201901010101,3,4,4,ラグリマスネグラス,牡2,51.0,団野大成,1:50.9,5,46.6,6.0,546(+6),[東] 藤沢和雄
201901010101,4,8,9,キタノコドウ,牡2,51.0,菅原明良,1:51.5,3.1/2,56.8,7.0,458(-8),[東] 高木登
201901010101,5,5,5,ネモフィラブルー,牡2,54.0,川島信二,1:51.7,1.1/2,140.3,9.0,436(0),[西] 矢作芳人
...,...,...,...,...,...,...,...,...,...,...,...,...,...
201910020812,10,7,12,オーシャンスケイプ,牝3,52.0,大野拓弥,1:49.8,3/4,45.7,10.0,414(+6),[西] 西村真幸
201910020812,11,3,3,プリヴェット,牝4,55.0,藤岡康太,1:50.3,3,29.8,9.0,486(+2),[西] 今野貞一
201910020812,12,5,7,アドマイヤクィーン,牝4,55.0,松山弘平,1:50.3,クビ,29.6,8.0,476(+6),[西] 須貝尚介
201910020812,13,8,13,タムロドリーム,牝3,49.0,亀田温心,1:50.4,1/2,134.8,12.0,472(+2),[西] 西園正都
